## Jeopardy! Data Manipulation with Pandas

This real-world, completely unmodified dataset of *Jeopardy!* questions and answers was provided by Codecademy as part of a data analytics course. *Jeopardy!* is a trivia show in which the the question is given as a descriptive clue and participants answer with an actual question. For example:

**Question**: This Python package helps data scientists inspect, manipulate, and analyze data efficiently.
**Answer**: What is Pandas?

Each question and answer are associated with a category and a dollar reward value as well as a date and number identifying the show in which it was used. Further, each question is part of a round, either *Jeopardy!* or *Double Jeopardy!*, where *Double Jeopardy!* questions have twice the amount of value as those in the *Jeopardy!* round.

While the prompts were given by Codecademy, all of the code and text are my own thinking and writing.

To review sample outputs, please uncomment (remove '##') in front of the print statements. Then, click 'Run' in the toolbar above or hit `Ctrl+Enter` on your keyboard.

### Load dataset and inspect data

In [20]:
#Import pandas library
import pandas as pd

#Ensure full contents of columns will be displayed
pd.set_option('display.max_colwidth', None)

#Load csv file into DataFrame
df = pd.read_csv('jeopardy.csv')

#Inspect data
##print(df.head())

### Clean up column names

In [21]:
#Rename columns for easier coding in analysis of columns
df.rename(columns={
  'Show Number': 'show_number',
  ' Air Date': 'air_date',
  ' Round': 'round',
  ' Category': 'category',
  ' Value': 'value',
  ' Question': 'question',
  ' Answer': 'answer'}, inplace=True)

#Inspect new column names by looking at them in one list
column_names = df.columns.values.tolist()
##print(column_names)

### Filter dataset for a given list of words

While the category column is helpful to study general topics, it would be interesting to dig a little deeper and be able to explore hunches about keywords that may appear more frequently on *Jeopardy!* than others. Perhaps questions are skewed towards subjects of European American, European, or English culture. For someone participating on the show, a similar analysis could guide preparation efforts.

In [17]:
#Function returning a new dataframe of a specified column whose values contain one or more words in the list
def filter_by_words_values(dataframe, col, lst):
  lst = [item.lower() for item in lst]
  new_df = pd.DataFrame(columns = [col])
  for i in dataframe[col]:
    i = i.lower()
    for el in lst:
      if el in i:
        new_row = pd.DataFrame([i], columns=[col])
        new_df = pd.concat([new_row,new_df],  axis=0, ignore_index=True)
        break
  return new_df

#Function returning the number of values in a specified column that contain one or more words in the list
def filter_by_words_number(dataframe, col, lst):
  lst = [item.lower() for item in lst]
  new_df = pd.DataFrame(columns = [col])
  for i in dataframe[col]:
    i = i.lower()
    for el in lst:
      if el in i:
        new_row = pd.DataFrame([i], columns=[col])
        new_df = pd.concat([new_row,new_df],  axis=0, ignore_index=True)
        break
  return len(new_df)

#Testing functions for 'question' column
lst1 = ['Europe', 'European']
lst2 = ['Asia', 'Asian']
lst3 = ['Africa', 'African']
##print(filter_by_words_values(df, 'question', lst1))
##print(filter_by_words_number(df, 'question', lst2))
##print(filter_by_words_number(df, 'question', lst3))

#### Reflection

After building the functions above, I thought of two other features I could have created. 

I thought about how I could make sure that the functions will only consider the word in itself and not as a substring of another word (e.g. 'story' in 'history'). I would find the start and end indeces of `el` and `i`. If the index, if it exists, before and after `el` in `i` is **not** a letter of the alphabet, `i` would get added to the resulting DataFrame, otherwise not.

It could also be very interesting to have a function return the column values that contain *all* of the words in the list passed in. I would remove the loop iterating through each list item, change the if-condition to `if all(el in i for el in lst):`, and remove the `break` statement to do this.

### Convert 'value' column to floats in order to perform calculations

One major incentive to do well on *Jeopardy!* is money! If we want to find out which topics and keywords are associated with higher rewards than others, we need to consult the 'value' column along with questions and answers. This analysis could again uncover patterns or help a participant prepare for the most rewarding questions. The current 'value' column contains $ signs and decimal places, making it impossible to perform calculations with this column's data.

In [4]:
#Build function converting values with `$` signs and/or decimal places to floats
def str_to_float(x):
  if x == 'None':
    return 0
  else:
   x = x.strip("$")
   x = x.replace(",", "")
   x = float(x)
   return x

#Create a new column for the float values called number_values
df['number_value'] = df.value.apply(str_to_float)

#Test results
##print(df['number_value'])

### Function to obtain average value of questions containing a word

In [5]:
def avg_value_of_words(dataframe, col, word):
  dataframe[col] = dataframe[col].apply(str.lower)
  filtered_qs =  filter_by_words_values(dataframe, col, [word])
  merged_df = pd.merge(filtered_qs, dataframe)
  return round(merged_df['number_value'].mean(), 2)

#To test function, compare average value of questions containing 'German' to those containing 'English'
##print(avg_value_of_words(df, 'question', "German"))
##print(avg_value_of_words(df, 'question', "English"))

### Compare unique categories and their frequency by round (Jeopardy! vs. Double Jeopardy!)

The relative relationship between topics and reward can also be studied with the data of the 'round' column.

In [23]:
def category_and_round(dataframe, cat):
  cat = cat.lower()
  singlej = dataframe[dataframe['round'] == 'Jeopardy!']
  doublej = dataframe[dataframe['round'] == 'Double Jeopardy!']
  singlej_count = filter_by_words_number(singlej, 'category', [cat])
  doublej_count = filter_by_words_number(doublej, 'category', [cat])
  return singlej_count, doublej_count

#print(category_and_round(df, 'history'))

#### Reflection

One way to dig deeper into this analysis could be to search by specific keywords in questions rather than categories. The filter functions above would help perform this task.

One point of caution here is that categories that frequently appear in the double category might not necessarily be the most lucrative questions. What is the actual size of the values that are doubled? How does their mean compare to that of other categories or keywords?
In turn, `category_and_round` gives us food for thought on the interpretations of the `avg_value_of_words` function. `avg_value_of_words`, as I wrote it, does not take the round into consideration. I could add a column of `total_value`, where `value` gets doubled if the round is 'Double Jeopardy!' to explore this relationship. This would allow me to modify `avg_value_of_words` to adjust for the round, rendering it a more helpful function to study dollar amounts than `category_and_round`. The results of `category_and_round` still provide interesting information on the kinds of prompts that are more likely to appear in a 'Double Jeopardy!' round and are thus subject to more excitement and pressure on the show.

### Function that returns the count of each unique answer to questions containing a certain word

General topics such as 'Ancient History' or 'English Literature' are helpful to get an idea of the question/answer pair that is about to be prompted. Furthermore, it would be interesting to explore whether *Jeopardy!* questions of a certain category tend to gravitate around very specific figures, places, art pieces, etc. While certainly a helpful preparation tool, this function could help show writers diversify their prompts by learning which topics may have been over-used in the past. As a result, the show would become less predictable, more inclusive, and more challenging since doing well will require a broader range of knowledge.

In [15]:
def unique_answer_count(dataframe, col1, col2, word):
    dataframe[col1] = dataframe[col1].astype(str)
    dataframe[col2] = dataframe[col2].astype(str)
    dataframe[col1] = dataframe[col1].apply(str.lower)
    dataframe[col2] = dataframe[col2].apply(str.lower)
    word = word.lower()
    filtered_qs =  filter_by_words_values(dataframe, col1, [word])
    merged_df = pd.merge(filtered_qs, dataframe)
    return merged_df[col2].value_counts()

#Test function
##print(unique_answer_count(df, 'question', 'answer', 'English'))

### Examine frequency of a word in a column by a given decade

Has *Jeopardy!* evolved with the tides of time and thus stayed relevant for the knowledge carried by multiple generations? As a result, would it be more fruitful to study more recent datasets than older ones?

In [22]:
def filter_by_word_and_decade(dataframe, col1, col2, word, decade):
  word = word.lower()
  dataframe[col1] = dataframe[col1].apply(str.lower)
  dataframe[col2] = dataframe[col2].apply(lambda x: x[0:3])
  new_df = dataframe[(dataframe[col2] == decade[0:3]) & (dataframe[col1].str.contains(word))]
  return len(new_df)

##print(filter_by_word_and_decade(df, 'question', 'air_date', "Computer", '2000'))
##print(filter_by_word_and_decade(df, 'question', 'air_date', "Computer", '1990'))